In [1]:
import os
from glob import glob
from pyvrs import SyncVRSReader

In [2]:
base_dir = "aea_main"
output_base_dir = "aea_main_rgb"
os.makedirs(output_base_dir, exist_ok=True)
vrs_files = glob(os.path.join(base_dir, '*/recording.vrs'))

In [ ]:
for vrs_file in vrs_files:
    sequence_name = os.path.basename(os.path.dirname(vrs_file))
    output_dir = os.path.join(output_base_dir, sequence_name)
    os.makedirs(output_dir, exist_ok=True)

    reader = SyncVRSReader(vrs_file)
    stream_id = '214-1'
    last_unique_timestamp = None
    record_index = 0
    image_count = 0

    while True:
        record = reader.read_next_record(stream_id, 'data', record_index)
        if record is None:
            break  # End of data stream

        current_timestamp = record.timestamp
        if current_timestamp == last_unique_timestamp:
            # This is a duplicate timestamp; skip to the next record
            record_index += 1
            continue

        # New unique timestamp found; update tracking variable
        last_unique_timestamp = current_timestamp

        # Process the record's image data
        image_blocks = record.image_blocks
        if image_blocks:
            output_file_path = os.path.join(output_dir, f"{sequence_name}_image_{image_count:06}.jpg")
            with open(output_file_path, 'wb') as f:
                image_blocks[0].tofile(f)  # Assuming the first block is the target
            image_count += 1

        record_index += 1

    print(f"Completed processing {sequence_name}, total unique images: {image_count}.")